# Load text data

In [2]:
import pandas as pd

# load the dataset
df = pd.read_csv('./Movie.csv', skipinitialspace=True) # encoding='latin'

# random state
rs = 42

# as usual, explore the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 21 columns):
Cast 1          799 non-null object
Cast 2          799 non-null object
Cast 3          799 non-null object
Cast 4          799 non-null object
Cast 5          799 non-null object
Cast 6          799 non-null object
Description     799 non-null object
Director 1      799 non-null object
Director 2      799 non-null object
Director 3      799 non-null object
Genre           799 non-null object
Rating          799 non-null object
Release Date    799 non-null object
Runtime         799 non-null object
Studio          799 non-null object
Title           799 non-null object
Writer 1        799 non-null object
Writer 2        799 non-null object
Writer 3        799 non-null object
Writer 4        799 non-null object
Year            799 non-null int64
dtypes: int64(1), object(20)
memory usage: 131.2+ KB


# Get Description and avg length (not needed for assignment)

In [4]:
#import warnings
#warnings.filterwarnings("ignore")
# print out the first 200 characters of the first row of text column
#print(df.get_value(index=0, col='Description')[:200])

In [6]:
# average length of text column
print(df['Description'].apply(lambda x: len(x)).mean())

761.7859824780976


# Data Prepocessing

In [9]:
#Import data preprocessing libraries
import string
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag

# initialise WordNet lemmatizer and punctuation filter
lemmatizer = WordNetLemmatizer()
punct = set(string.punctuation)

# load the provided stopwords (DO WE NEED TO FIND MOVIE RELATED STOPWORDS?)
#df_stop = pd.read_csv('./Movie.csv', skipinitialspace=True) # encoding='latin'

# Get stopwords with the default NLTK English stopwords
stopwords = set(sw.words('english'))

def lemmatize(token, tag):
    tag = {
        'N': wn.NOUN,
        'V': wn.VERB,
        'R': wn.ADV,
        'J': wn.ADJ
    }.get(tag[0], wn.NOUN)

    return lemmatizer.lemmatize(token, tag)

Cab tokenizer function

In [10]:
def cab_tokenizer(document):
    # initialize token list
    tokens = []
    
    # split the document into sentences
    for sent in sent_tokenize(document):
        # split the document into tokens and then create part of speech tag for each token
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # preprocess and remove unnecessary characters
            token = token.lower()
            token = token.strip()
            token = token.strip('_')
            token = token.strip('*')

            # If stopword, ignore token and continue
            if token in stopwords:
                continue

            # If punctuation, ignore token and continue
            if all(char in punct for char in token):
                continue

            # Lemmatize the token and add back to the tokens list
            lemma = lemmatize(token, tag)
            tokens.append(lemma)
    
    return tokens

# Vectorize movie descriptions

Variables included: Description, Genre, Title, Writer because they may be relevant to the description (elaborate),
Variables excluded: ...

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
df['Genre'].unique()
# tf idf vectoriser
#tfidf_vec = TfidfVectorizer(tokenizer=cab_tokenizer, ngram_range=(1,2))
#X = tfidf_vec.fit_transform(df['Description'] + " " + df["Title"] + " " + df["Genre"]) # derive genre from clusters

# see the number of unique tokens produced by the vectorizer. Lots of them...
#print(len(tfidf_vec.get_feature_names()))

array(['Documentary', 'SciFi', 'Kids&Family', 'Romance'], dtype=object)